In [1]:
import torch
import torch.nn.functional as F

import numpy as np
import cv2
import os
import sys
sys.path.append('..')


# Slowfast imports
from slowfast.models import build_model
from slowfast.utils.parser import load_config, alt_parse_args
from slowfast.datasets.loader import construct_loader



/home/kukushkin/anaconda3/envs/slowfast/lib/python3.10/site-packages/torchvision-0.17.2+c1d70fe-py3.10-linux-x86_64.egg/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/kukushkin/anaconda3/envs/slowfast/lib/python3.10/site-packages/torchvision-0.17.2+c1d70fe-py3.10-linux-x86_64.egg/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [3]:
save_dir = "/home/kukushkin/imslowfast/cam_videos/"
path_to_config = "/home/kukushkin/imslowfast/configs/SLOW_8x8_R50_Local.yaml"
path_to_ckpt = "/home/kukushkin/imslowfast/weights/checkpoint_epoch_00200.pyth"

args = alt_parse_args()[:-1]
cfg = load_config(
    args[0],
    path_to_config=path_to_config,
)
checkpoint = torch.load(path_to_ckpt, map_location="cpu")

model = build_model(cfg)
model.load_state_dict(checkpoint["model_state"])
model.eval()
model.cpu()

ResNet(
  (s1): VideoModelStem(
    (pathway0_stem): ResNetBasicStem(
      (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
      (bn): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (pool_layer): MaxPool3d(kernel_size=[1, 3, 3], stride=[1, 2, 2], padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
  )
  (s2): ResStage(
    (pathway0_res0): ResBlock(
      (branch1): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (branch1_bn): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (branch2): BottleneckTransform(
        (a): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (a_bn): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (a_relu): ReLU(inplace=True)
        (b): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), 

In [4]:
classifier = model.head.projection

loader = construct_loader(cfg, "test") 
sample = next(iter(loader))

inputs, labels, index, time, meta = sample

SAMPLING.BALANCED: False; BALANCE_TYPE: None


/home/kukushkin/imslowfast/notebooks/../slowfast/datasets/decoder.py:335: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1711403380909/work/torch/csrc/utils/tensor_numpy.cpp:206.)
  video_tensor = torch.from_numpy(np.frombuffer(video_handle, dtype=np.uint8))
/home/kukushkin/imslowfast/notebooks/../slowfast/datasets/decoder.py:335: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppres

In [5]:
meta["video_name"]

['59737670.mp4',
 '59737571.mp4',
 '34030610.mp4',
 'acp000bckb.mp4',
 '36070482.mp4',
 '36070477.mp4',
 '36070473.mp4',
 '36070472.mp4']

In [6]:
labels

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]],
       dtype=torch.float64)

In [7]:
inputs[0].shape

torch.Size([8, 3, 16, 256, 256])

In [8]:
feature_maps = model.s5(
    model.s4(model.s3(model.s2(model.s1([inputs[0]]))))
)[0]

In [9]:
feature_maps.shape

torch.Size([8, 2048, 16, 8, 8])

In [10]:
# Get original video
video_path = "/home/kukushkin/imslowfast/dataset/videos/"

batch_frames = []

for path in meta["video_name"]:
    cap = cv2.VideoCapture(os.path.join(video_path, path))
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (256, 256))
        frames.append(frame)
        

    batch_frames.append(np.stack(frames))

batch_frames = np.stack(batch_frames)

In [16]:
def returnCAM(feature_conv, weight_softmax, class_idx):
    # add first dimension
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape
    
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[idx].dot(feature_conv.reshape((nc, h * w)))
        cam = cam.reshape(h, w)
        #cam = cam - np.min(cam)
        #cam_img = cam / np.max(cam)
        #cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam, size_upsample))
    return output_cam


def returnCAM_batch(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape
    output_cam = []

    # Reshape feature_conv to (bz, nc, h*w) for batch processing
    feature_conv_reshaped = feature_conv.reshape((bz, nc, h * w))

    for idx in class_idx:
        # Compute CAM for all samples in the batch
        cam = np.dot(weight_softmax[idx], feature_conv_reshaped)
        cam = cam.reshape(bz, h, w)
        # Apply min-max normalization across all samples in the batch
        cam_min = cam.min(axis=(1, 2), keepdims=True)
        cam_max = cam.max(axis=(1, 2), keepdims=True)
        cam_normalized = (cam - cam_min) / (cam_max - cam_min)

        # Convert to uint8 and resize
        cam_img = np.uint8(255 * cam_normalized)
        output_cam.append(np.array([cv2.resize(img, size_upsample) for img in cam_img]))

    # Stack the results for each class
    return np.stack(output_cam, axis=1)

In [17]:
batch_frames.shape

(8, 360, 256, 256, 3)

In [18]:
feature_maps.shape

torch.Size([8, 2048, 16, 8, 8])

In [19]:
cams = []

for i in range(feature_maps.shape[0]):
    video_cam = []   
    for beh_idx in range(14):
        beh_cam = []
            
        for f in range(feature_maps.shape[2]):
            spatial_map = feature_maps[i, :, f, :, : ].detach().numpy()
            spatial_map = np.expand_dims(spatial_map, axis=0)
            frame_cam = returnCAM(spatial_map, classifier.weight.detach().numpy(), [beh_idx])
            beh_cam.append(frame_cam)

        # normalize
        beh_cam = np.stack(beh_cam)
        beh_cam = beh_cam - np.min(beh_cam) / np.max(beh_cam)
        # convert to uint8
        beh_cam = np.uint8(255 * beh_cam)
        video_cam.append(beh_cam)

    cams.append(np.stack(video_cam))


cams = np.stack(cams)


In [20]:
cams.shape

(8, 14, 16, 1, 256, 256)

In [24]:
cams.shape

(8, 14, 16, 1, 256, 256)

In [25]:
batch_frames.shape

(8, 360, 256, 256, 3)

In [26]:
cams.shape

(8, 14, 16, 1, 256, 256)

In [27]:
# to torch 
tensor_cams = torch.tensor(cams)
# convert to float
tensor_cams = tensor_cams.float()

tensor_cams = torch.squeeze(tensor_cams)
print(tensor_cams.shape)

torch.Size([8, 14, 16, 256, 256])


In [29]:
output_tensor = F.interpolate(tensor_cams, size=(360, 256, 256), mode='trilinear', align_corners=False)
#  convert tensor to uint8
output_tensor = output_tensor.byte()
output = output_tensor.detach().numpy()
output_tensor.shape

torch.Size([8, 14, 360, 256, 256])

In [30]:
batch_frames.shape

(8, 360, 256, 256, 3)

In [31]:


# create directory if not exists
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [32]:
for bg_idx in range(batch_frames.shape[0]):
        video_cam = output[bg_idx]
        bg = batch_frames[bg_idx]
        for i in range(video_cam.shape[0]):
            # create video as mp4
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(os.path.join(save_dir, f"{meta['video_name'][bg_idx].split('.')[0]}_{bg_idx}_{i}.mp4"), fourcc, 24, (256, 256))
            results = []
            for j in range(video_cam.shape[1]):
                class_cam_frame = video_cam[i, j, ...]
                class_cam_frame = np.uint8(255 * class_cam_frame)

                heatmap = cv2.applyColorMap(class_cam_frame, cv2.COLORMAP_JET)
                result = heatmap * 0.3 + bg[i] * 0.5
                # convert to uint8
                result = np.uint8(result)

                out.write(result)

            out.release()

    
